In [25]:
import pandas as pd
import numpy as np
from typing import Iterable
from sklearn.preprocessing import LabelEncoder

In [26]:
def universal_one_hot_encoder(df: pd.DataFrame, target_col: str):
    data_to_encode = df.copy()

    # Определяем категориальные признаки (все, что не числа и не целевая колонка)
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    categorical_features = [
        col for col in data_to_encode.columns if col not in numeric_cols or col == target_col
    ]
    if target_col in categorical_features:
        categorical_features.remove(target_col)

    # Применяем One-Hot Encoding
    data_ohe = pd.get_dummies(
        data_to_encode,
        columns=categorical_features,
        dummy_na=False,
        dtype=int
    )

    # Собираем список всех дискретных колонок для CTGAN
    # Это все колонки, которых не было в исходном списке числовых колонок
    discrete_features_ohe = [
        col for col in data_ohe.columns if col not in numeric_cols or col == target_col
    ]

    return data_ohe, discrete_features_ohe

In [27]:
def universal_drop_nans(df: pd.DataFrame, patterns: Iterable[str] | str | None = None, regex: bool = False) -> pd.DataFrame:
    """
    Заменяет указанные паттерны на NaN и возвращает новую таблицу с удалёнными строками, содержащими NaN.
    - patterns: строка или итерация строк; если None, просто выполняется dropna().
    - regex: если True, паттерны трактуются как регулярные выражения.
    """
    df_copy = df.copy()
    if patterns is None:
        return df_copy.dropna()
    if isinstance(patterns, str):
        patterns = [patterns]
    # replace поддерживает список значений; используем параметр regex при необходимости
    df_copy.replace(to_replace=list(patterns), value=np.nan, inplace=True, regex=regex)
    return df_copy.dropna()

In [28]:
def universal_impute_median(
    df: pd.DataFrame,
    target_col: str,
    threshold: float = 0.1
) -> pd.DataFrame:
    """
    Заменяет NaN медианой (для числовых) или модой (для категориальных),
    если доля пропусков не превышает threshold.

    Parameters:
    - df: исходный DataFrame
    - target_col: имя целевой колонки (не будет обрабатываться)
    - threshold: максимальная доля пропусков для замены (по умолчанию 0.5 = 50%)

    Returns:
    - DataFrame с заполненными NaN
    """
    df_copy = df.copy()

    # Числовые колонки без target
    numeric_cols = df_copy.select_dtypes(include=np.number).columns.tolist()
    if target_col in numeric_cols:
        numeric_cols.remove(target_col)

    # Категориальные колонки без target
    categorical_cols = df_copy.select_dtypes(exclude=np.number).columns.tolist()
    if target_col in categorical_cols:
        categorical_cols.remove(target_col)

    # Заполнение числовых колонок медианой
    for col in numeric_cols:
        nan_count = df_copy[col].isna().sum()
        if nan_count == 0:
            continue
        nan_ratio = nan_count / len(df_copy)

        if nan_ratio <= threshold:
            median_value = df_copy[col].median()
            df_copy[col].fillna(median_value, inplace=True)
            print(f"Заполнено {nan_count} пропусков в '{col}' медианой {median_value:.2f}")

    # Заполнение категориальных колонок модой
    for col in categorical_cols:
        nan_count = df_copy[col].isna().sum()
        if nan_count == 0:
            continue
        nan_ratio = nan_count / len(df_copy)

        if nan_ratio <= threshold:
            mode_value = df_copy[col].mode()[0] if not df_copy[col].mode().empty else None
            if mode_value is not None:
                df_copy[col].fillna(mode_value, inplace=True)
                print(f"Заполнено {nan_count} пропусков в '{col}' модой '{mode_value}'")

    return df_copy

In [29]:
def universal_label_encoder(df: pd.DataFrame, target_col: str) -> pd.DataFrame:
    df_copy = df.copy()
    le = LabelEncoder()
    df_copy[target_col] = le.fit_transform(df_copy[target_col].astype(str))
    return df_copy

In [30]:
data = pd.read_csv('../data/unprocessed/mushroom.csv')
target_col = 'habitat'
data = universal_impute_median(data, target_col)
data = universal_drop_nans(data, ['?'])
data, discrete_features_ohe = universal_one_hot_encoder(data, target_col)
data = universal_label_encoder(data, target_col)
data.to_csv('../data/processed/mushroom_processed.csv', index=False)

In [31]:
data

,habitat,class_e,class_p,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,...,spore-print-color_r,spore-print-color_u,spore-print-color_w,spore-print-color_y,population_a,population_c,population_n,population_s,population_v,population_y
41,0,1,0,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,1
57,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
60,6,1,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
83,4,0,1,0,0,0,0,0,1,1,...,1,0,0,0,1,0,0,0,0,0
115,4,0,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25836,0,1,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
25875,4,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
25894,1,1,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
25938,5,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [32]:
discrete_features_ohe

['habitat',
 'class_e',
 'class_p',
 'cap-shape_b',
 'cap-shape_c',
 'cap-shape_f',
 'cap-shape_k',
 'cap-shape_s',
 'cap-shape_x',
 'cap-surface_f',
 'cap-surface_g',
 'cap-surface_s',
 'cap-surface_y',
 'cap-color_b',
 'cap-color_c',
 'cap-color_e',
 'cap-color_g',
 'cap-color_n',
 'cap-color_p',
 'cap-color_r',
 'cap-color_u',
 'cap-color_w',
 'cap-color_y',
 'ruises_f',
 'ruises_t',
 'odor_a',
 'odor_c',
 'odor_f',
 'odor_l',
 'odor_m',
 'odor_n',
 'odor_p',
 'odor_s',
 'odor_y',
 'gill-attachment_a',
 'gill-attachment_f',
 'gill-spacing_c',
 'gill-spacing_w',
 'gill-size_b',
 'gill-size_n',
 'gill-color_b',
 'gill-color_e',
 'gill-color_g',
 'gill-color_h',
 'gill-color_k',
 'gill-color_n',
 'gill-color_o',
 'gill-color_p',
 'gill-color_r',
 'gill-color_u',
 'gill-color_w',
 'gill-color_y',
 'stalk-shape_e',
 'stalk-shape_t',
 'stalk-root_b',
 'stalk-root_c',
 'stalk-root_e',
 'stalk-root_r',
 'stalk-surface-above-ring_f',
 'stalk-surface-above-ring_k',
 'stalk-surface-above-ring_